In [1]:
from ydata_profiling import ProfileReport
import pandas as pd

In [2]:
df = pd.read_parquet(r"../data/types_fixed_german_credit_data.parquet", engine="pyarrow")
df["Job"] = pd.Categorical(df["Job"], categories=[0,1,2,3], ordered=True)

In [3]:
report = ProfileReport(df.sample(frac=0.80), explorative=True)
report.to_file(r"../data/eda.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]